# Live Coding Exercise
*Module 7: Database Queries*

Created: 03/15/23 by Tom Lever

Updated: 03/15/23 by Tom Lever

ER Diagram: https://dbdocs.io/jkropko/NBADB?view=relationships

In [24]:
import numpy as np
import pandas as pd
import sqlite3
nba_db = sqlite3.connect("nba.db")
# Player's individual stats for all games in the 2022 season, along with the player's name
year = '2022'
myquery = f'''
SELECT *
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE g.season = {year}
'''
pd.read_sql_query(myquery, nba_db)

,game_id,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,...,sf%,pf%,c%,active_position_minutes,player_id,player,game_id,game_date,ot,season
0,202202170BRK,aldrila01,0,28:19,5,8,0.625,0,0,0.000,...,0.0,0.0,100.0,70.877559,aldrila01,LaMarcus Aldridge,202202170BRK,2022-02-17,0,2022
1,202202170BRK,avdijde01,0,30:50,4,10,0.400,1,3,0.333,...,62.0,31.0,0.0,54.944529,avdijde01,Deni Avdija,202202170BRK,2022-02-17,0,2022
2,202202170BRK,brownbr01,1,34:57,3,10,0.300,0,1,0.000,...,58.0,11.0,0.0,43.397040,brownbr01,Bruce Brown,202202170BRK,2022-02-17,0,2022
3,202202170BRK,bryanth01,1,14:04,5,6,0.833,0,1,0.000,...,0.0,0.0,100.0,36.472537,bryanth01,Thomas Bryant,202202170BRK,2022-02-17,0,2022
4,202202170BRK,caldwke01,1,25:26,3,7,0.429,1,3,0.333,...,67.0,0.0,0.0,47.021807,caldwke01,Kentavious Caldwell-Pope,202202170BRK,2022-02-17,0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23856,202201210PHI,niangge01,0,30:13,3,11,0.273,1,7,0.143,...,33.0,64.0,3.0,44.840107,niangge01,Georges Niang,202201210PHI,2022-01-21,0,2022
23857,202201210PHI,powelmy01,0,0:00,0,0,0.000,0,0,0.000,...,0.0,0.0,0.0,48.463595,powelmy01,Myles Powell,202201210PHI,2022-01-21,0,2022
23858,202201210PHI,sprinja01,0,0:00,0,0,0.000,0,0,0.000,...,63.0,0.0,0.0,36.448977,sprinja01,Jaden Springer,202201210PHI,2022-01-21,0,2022
23859,202201210PHI,winslju01,0,0:00,0,0,0.000,0,0,0.000,...,8.0,72.0,20.0,62.753626,winslju01,Justise Winslow,202201210PHI,2022-01-21,0,2022


In [25]:
# Determine who should have been MVP (is it Jokic?)
# Highest FG % across a whole season: sum of fg and fga in the season
myquery = '''
SELECT p.player, SUM(pg.fg) AS fg, SUM(pg.fga) AS fga, 1.0*fg/fga AS fg_pct
FROM player_game pg
INNER JOIN players p
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE g.season = 2022
GROUP BY pg.player_id, p.player
HAVING SUM(pg.fga) > 100
ORDER BY fg_pct DESC
'''
pd.read_sql_query(myquery, nba_db)

,player,fg,fga,fg_pct
0,Luka Garza,55,124,1.0
1,Josh Hart,219,426,1.0
2,Aaron Holiday,109,227,1.0
3,Damian Jones,90,141,1.0
4,Zeke Nnaji,95,176,1.0
...,...,...,...,...
390,Xavier Tillman,69,153,NaN
391,Xavier Tillman Sr.,69,153,NaN
392,Duane Washington Jr.,91,241,NaN
393,Justise Winslow,92,204,NaN


Point = 1

3PM = 1

FGA = -1

FGM = 2

FTA = -1

FTM = 1

REB = 1

AST = 2

STL = 4

BLK = 4

TOV = -2

In [21]:
myquery = '''
SELECT
    p.player,
    SUM(pg.pts) AS pts,
    SUM(pg.fg3) AS fg3,
    SUM(pg.fg) AS fg,
    SUM(pg.fga) AS fga,
    SUM(pg.fta) AS fta,
    SUM(pg.ft) AS ft,
    SUM(pg.trb) AS trb,
    SUM(pg.ast) AS ast,
    SUM(pg.stl) AS stl,
    SUM(pg.blk) AS blk,
    SUM(pg.tov) AS tov,
    SUM(pg.pts) + SUM(pg.fg3) + 2*SUM(pg.fg) - SUM(pg.fga) - SUM(pg.fta) + SUM(pg.ft) + SUM(pg.trb) + 2*SUM(pg.ast) + 4*SUM(pg.stl) + 4*SUM(pg.blk) - 2*SUM(pg.tov) AS mvp_points
FROM player_game pg
INNER JOIN players p
ON pg.player_id = p.player_id
INNER JOIN games g
ON pg.game_id = g.game_id
WHERE g.season = 2022
GROUP BY pg.player_id, p.player
ORDER BY mvp_points DESC
'''
pd.read_sql_query(myquery, nba_db)

,player,pts,fg3,fg,fga,fta,ft,trb,ast,stl,blk,tov,mvp_points
0,Nikola Jokic,1352,84,516,906,292,236,718,410,69,39,197,3082
1,Giannis Antetokounmpo,1443,58,496,906,545,393,551,296,48,68,162,2718
2,Dejounte Murray,1036,67,428,932,149,113,432,486,106,17,125,2637
3,Chris Paul,866,60,325,667,185,156,259,619,108,18,138,2605
4,Trae Young,1475,158,495,1085,364,327,206,493,53,5,217,2491
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,Jaylen Morris,2,0,0,6,4,2,2,2,0,0,1,-2
635,Jaysean Paige,0,0,0,3,0,0,1,1,0,0,1,-2
636,Zylan Cheatham,0,0,0,3,0,0,0,0,0,0,0,-3
637,Nate Hinton,0,0,0,1,0,0,0,0,0,0,1,-3


In [23]:
# win / loss
myquery = '''
SELECT t.game_id, t.team_abbrev, t.opponent_abbrev, t.team_score AS team_score, s.team_score AS opponent_score, (t.team_score > s.team_score) AS win 
FROM team_game t
INNER JOIN team_game s
ON t.team_abbrev = s.opponent_abbrev AND t.game_id = s.game_id
'''
pd.read_sql_query(myquery, nba_db)

,game_id,team_abbrev,opponent_abbrev,team_score,opponent_score,win
0,202202170BRK,WAS,BRK,117,103,1
1,202202170BRK,BRK,WAS,103,117,0
2,202202170CHO,MIA,CHO,111,107,1
3,202202170CHO,CHO,MIA,107,111,0
4,202202170LAC,HOU,LAC,111,142,0
...,...,...,...,...,...,...
6389,202002270GSW,GSW,LAL,86,116,0
6390,202002290PHO,GSW,PHO,115,99,1
6391,202003010GSW,GSW,WAS,110,124,0
6392,202003030DEN,GSW,DEN,116,100,1
